# LBM tutorial

## hello world 

 - First we will assume that there is a current version of a sailfish in current directory.
 
         !git clone https://github.com/sailfish-team/sailfish

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sys,os
sys.path.append('./sailfish/')


In [ ]:
!pwd

In [ ]:
ls -lrta 

In [ ]:
%%writefile ex1.py
import numpy as np
from sailfish.subdomain import Subdomain2D
from sailfish.node_type import NTFullBBWall, NTEquilibriumVelocity
from sailfish.controller import LBSimulationController
from sailfish.lb_single import LBFluidSim

class MyBlock(Subdomain2D):
    max_v = 0.1

    def boundary_conditions(self, hx, hy):
        wall_map = ( (hx == self.gx-1) | (hx == 0) | (hy == 0) )
        self.set_node( (hy == self.gy-1) & (hx>0) & (hx<self.gx-1) , NTEquilibriumVelocity((self.max_v, 0.0)) )
        self.set_node(wall_map, NTFullBBWall)
        
    def initial_conditions(self, sim, hx, hy):
        sim.rho[:] = 1.0

class MySim(LBFluidSim):
    subdomain = MyBlock

LBSimulationController(MySim).run()

In [ ]:
%%sh
export PYTHONPATH=$PWD/sailfish/
python ex1.py --max_iters=100 --lat_nx=32 --lat_ny=32

In [ ]:
%%sh
export PYTHONPATH=$PWD/sailfish/
python ex1.py --max_iters=1000 --every=1000 \
 --lat_nx=32 --lat_ny=48 \
 --visc=0.1\
 --output=ex1 --output_format=npy

In [ ]:
!ls ex1* -rtla

In [ ]:
data = np.load("ex1.0.1000.npz")
data.files

In [ ]:
data['v'][0].shape

In [ ]:
vx,vy = data['v']
vy.shape

In [ ]:
ny,nx = vx.shape

In [ ]:
plt.imshow(np.sqrt(vx**2+vy**2),origin='top')

In [ ]:
Y,X = np.mgrid[0:ny,0:nx]

In [ ]:
plt.figure(figsize=(5,5))
plt.xlim(1,nx-1)
plt.ylim(1,ny-1)
plt.quiver(X,Y,vx,vy)

In [ ]:
vx.shape,X.shape

In [ ]:
plt.figure(figsize=(5,ny/nx*5))
plt.xlim(1,nx-2)
plt.ylim(1,ny-1)
plt.contourf(X,Y,np.sqrt(vy**2+vx**2),256)

In [ ]:
plt.figure(figsize=(5,ny/nx*5))
plt.xlim(1,nx-2)
plt.ylim(1,ny-1)
plt.contourf(X,Y,np.sqrt(vy**2+vx**2),256)
plt.streamplot(X,Y,vx,vy,color='white')


In [ ]:
np.mgrid[0:4,0:4][0]

In [ ]:
x = np.linspace(0,3,4)
X,Y = np.meshgrid(x,x)
X

In [ ]:
%%writefile ex2.py
import numpy as np
from sailfish.subdomain import Subdomain2D
from sailfish.node_type import NTFullBBWall, NTEquilibriumVelocity
from sailfish.controller import LBSimulationController
from sailfish.lb_single import LBFluidSim

class MyBlock(Subdomain2D):
    max_v = 0.1

    def boundary_conditions(self, hx, hy):
        wall_map = ( (hx == self.gx-1) | (hx == 0) | (hy == 0) )
        self.set_node( (hy == self.gy-1) & (hx>0) & (hx<self.gx-1) , NTEquilibriumVelocity((self.max_v, 0.0)) )
        self.set_node(wall_map, NTFullBBWall)
        np.savez("hx.npz",hx=hx,hy=hy)
    def initial_conditions(self, sim, hx, hy):
        sim.rho[:] = 1.0
        
class MySim(LBFluidSim):
    subdomain = MyBlock

LBSimulationController(MySim).run()

In [ ]:
%%sh
export PYTHONPATH=$PWD/sailfish/
python ex2.py --max_iters=100 --lat_nx=32 --lat_ny=32

In [ ]:
!ls -lta hx*

In [ ]:
data = np.load("hx.npz")
hx = data['hx']
hy = data['hy']

In [ ]:
hx

In [ ]:
(hx>0)

In [ ]:
plt.imshow(hx>4)

In [ ]:
plt.imshow( (hy>30)|(hx<1) | (hx>30),interpolation='nearest',cmap='gray') 

In [ ]:
plt.imshow( (hx-10)**2+(hy-28)**2<6.2**2,interpolation='nearest',cmap='gray',origin='top') 

In [ ]:
x0,y0,a = 15,8,4
plt.imshow( (hx>x0) & (hx<x0+a) & (hy>y0) & (hy<y0+a),interpolation='nearest') 

In [ ]:
hx[1:-1,1:-1] = 2
plt.imshow(hx)

In [ ]:
%%writefile ex3.py
import numpy as np
from sailfish.subdomain import Subdomain2D
from sailfish.node_type import NTFullBBWall, NTEquilibriumVelocity
from sailfish.controller import LBSimulationController
from sailfish.lb_single import LBFluidSim

class MyBlock(Subdomain2D):

    def boundary_conditions(self, hx, hy):
        wall_map = ( (hx == self.gx-1) | (hx == 0) | (hy == 0) | (hy == self.gy-1) )
        self.set_node(wall_map, NTFullBBWall)
        x0,y0,a = 15,8,4
        square_map = (hx>x0) & (hx<x0+a) & (hy>y0) & (hy<y0+a)
        self.set_node( square_map , NTEquilibriumVelocity(( 0.1, 0.18)) )
        x0,y0,a = 15+6,8+6,4
        square_map = (hx>x0) & (hx<x0+a) & (hy>y0) & (hy<y0+a)
        self.set_node(square_map, NTFullBBWall)
        
    def initial_conditions(self, sim, hx, hy):
        sim.rho[:] = 1.0
        #sim.vx[hy == self.gy-1] = self.max_v


class MySim(LBFluidSim):
    subdomain = MyBlock

LBSimulationController(MySim).run()

In [ ]:
%%sh
export PYTHONPATH=$PWD/sailfish/
python ex3.py --max_iters=1000 --every=1000 \
 --lat_nx=32 --lat_ny=32 \
 --visc=0.1002 \
 --output=ex3 --output_format=npy

In [ ]:
!ls -lrta ex3*


In [ ]:
data = np.load("ex3.0.1000.npz")
vx,vy = data['v']
ny,nx = vx.shape
Y,X = np.mgrid[0:ny,0:nx]
plt.figure(figsize=(5,ny/nx*5))
plt.xlim(1,nx-2)
plt.ylim(1,ny-2)
plt.contourf(X,Y,np.sqrt(vy**2+vx**2),256)
plt.streamplot(X,Y,vx,vy,color='white')
